# Introduction

The documentation of this notebook is for those who are new to `python` and `polars`

combination of r_transaltion.py script and more_r_translation.py marimo notebook

for the second matching wrangle, the parts for specificEpithet and infraSpecificEpithet doesn't match with many results.

[Read the powers getting started guide](https://docs.pola.rs/user-guide/getting-started/)
You may check out the `Polars` API References and tutorials for more information.

String values are denoted with double quotations and contains unicode characters like alphabets, number and etc. eg: "test"

Booleans are binary values that are either 'True' or 'False'.

# Imports

In [ ]:
import marimo as mo
import polars as pl

`with_columns()`:[ Adds columns to this DataFrame. Added columns will replace existing columns with the same name.](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.with_columns.html)

`pl.col()`: [Create an expression representing column(s) in a DataFrame.](https://docs.pola.rs/api/python/stable/reference/expressions/col.html)

 `.str`: to get the string functions so that you could chain it with other funcitons such as `strip_chars()`

`strip_char()`: strip a certain characters that is present in the start or end of the string

In [ ]:
bos = (
    pl.read_csv("bos.csv")
    .rename(
        {
            "data cleanup changes": "data cleanup changes 1",
            "species": "specificEpithet",
            "subspecies": "infraspecificEpithet",
            "domain ": "domain",
        }
    )
    .with_columns(pl.col(pl.String).str.strip_chars(" "))
    .with_columns(genericName=pl.col("genus"))
)

`transpose()`: matrix transpose the tabular data.

In [ ]:
null_counts = (
    bos.null_count()
    .transpose(include_header=True)
    .rename({"column": "feature", "column_0": "len"})
)
columns_to_drop_as_all_nulls = (
    null_counts.filter(null_counts["len"] == bos.shape[0])
    .transpose()[0, :]
    .transpose()
    .to_series()
    .to_list()
)

In [ ]:
columns_to_drop_as_they_are_just_changes_logs = [
    "cleanup changes ",
    "Data cleanup changes",
    "data cleanup changes 1",
    "changes",
    "cleanup changes",
    "Unnamed: 18",
    "Unnamed: 17",
    "Unnamed: 19",
    "query",
    "query ",
    "issue",
    "cleanup changes/comments",
    "subphylum",
]

`~`: negation of the predicate expressions.
`fitler()` : [Filter rows, retaining those that match the given predicate expression(s). Only rows where the predicate resolves as True are retained; when the predicate result is False (or null), the row is discarded.](https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.filter.html)

`select()`: select specific columns

`when().then().otherwise()`: Like if else statement.
```
e.g. infraspecificEpithet=pl.when(pl.col("infraspecificEpithet").is_null())
        .then(pl.lit(""))
        .otherwise(pl.col("infraspecificEpithet"))
```
explaination: when values in infraspecificEpithet column is null then populate it with the literal empty string (as denoted as "") otherise populate it with infraspecificEpithet.


`lit()`: Tells the intepretator that you are refering to a literal string not any columns.

`pl.col("col1").is_in(["a","b"])`: is_in is used to check and filter in values that are the list of strings, these strings do not refer to any columns.

In [ ]:
bos_cleaned = (
    (
        bos.drop(columns_to_drop_as_all_nulls)
        .drop(columns_to_drop_as_they_are_just_changes_logs)
        .select(~pl.selectors.starts_with("Unnamed"))
    )
    .with_columns(
        infraspecificEpithet=pl.when(pl.col("infraspecificEpithet").is_null())
        .then(pl.lit(""))
        .otherwise(pl.col("infraspecificEpithet"))
    )
    .with_columns(
        infraspecificEpithet=pl.when(
            (pl.col("genericName") == "Glenea")
            & (pl.col("specificEpithet") == "mathemathica")
        )
        .then(pl.lit("mathematica"))
        .otherwise(pl.col("infraspecificEpithet"))
    )
    .with_columns(
        specificEpithet=pl.when(
            (pl.col("genericName") == "Glenea")
            & (pl.col("specificEpithet") == "mathemathica")
        )
        .then(pl.lit("mathematica"))
        .when(
            (pl.col("genericName") == "Bavia")
            & (pl.col("specificEpithet") == "sexupunctata")
        )
        .then(pl.lit("sexpunctata"))
        .when(
            (pl.col("genericName") == "Omoedus")
            & (pl.col("specificEpithet") == "ephippigera")
        )
        .then(pl.lit("ephippiger"))
        .when(
            (pl.col("genericName") == "Byblis")
            & (pl.col("specificEpithet") == "kallartha")
        )
        .then(pl.lit("kallarthra"))
        .when(
            (pl.col("genericName") == "Alcockpenaeopsis")
            & (pl.col("specificEpithet") == "hungerfordi")
        )
        .then(pl.lit("hungerfordii"))
        .when(
            (pl.col("genericName") == "Pseudosesarma")
            & (pl.col("specificEpithet") == "edwardsi")
        )
        .then(pl.lit("edwardsii"))
        .when(
            (pl.col("genericName") == "Urocaridella")
            & (pl.col("specificEpithet") == "antonbruuni")
        )
        .then(pl.lit("antonbruunii"))
        .when(
            (pl.col("genericName") == "Ocypode")
            & (pl.col("specificEpithet") == "cordimanus")
        )
        .then(pl.lit("cordimana"))
        .otherwise(pl.col("specificEpithet"))
    )
    .with_columns(
        domain=pl.lit("Eukarya"),
        kingdom=pl.when(
            (pl.col("phylum").is_in(["Ascomycota", "Basidiomycota"]))
        )
        .then(pl.lit("fungi"))
        .when(pl.col("phylum") == "Cyanobacteria")
        .then(pl.lit("Bacteria"))
        .when(pl.col("phylum") == "Ciliophora")
        .then(pl.lit("Protista"))
        .otherwise(pl.lit("Animalia")),
    )
    .with_columns(
        taxonName=pl.col("taxonName")
        .str.replace_all("<i>", "")
        .str.replace_all("</i>", ""),
    )
)

Lazyframes from `pl.scan_csv()` is use to efficiently process big csv files.

In [ ]:
_filter = pl.col("taxonomicStatus").is_in(
    [
        "accepted",
        "synonym",
        "homotypic synonym",
        "proparte synonym",
        "heterotypic synonym",
    ]
)
taxon_ranked_only = (
    pl.scan_csv(
        "gbif/Taxon.tsv",
        separator="\t",
        quote_char=None,
        cache=True,
    )
    .filter((pl.col("taxonRank") != "unranked") & _filter)
    .select(
        "taxonID",
        "genericName",
        "genus",
        "specificEpithet",
        "infraspecificEpithet",
        "taxonomicStatus",
        "acceptedNameUsageID",
        # "domain",
    )
    .filter((pl.col("genus") != "") & (pl.col("specificEpithet") != ""))
    .with_columns(
        infraspecificEpithet=pl.when(
            (pl.col("infraspecificEpithet").is_null())
        )
        .then(pl.lit(""))
        .otherwise(pl.col("infraspecificEpithet"))
    )
)

`cast()`: Cast column(s) in to a specific polars datatype.

`collect()`: Materialize this LazyFrame into a DataFrame. It is a slow function so use it sparingly and appropriately.

`is_duplicated()`: a expression function for checking if a row's set of values is duplicated from one of the previous row's set of values.

`fill_null()`: fills null values in a column with a particular string.

`list[0]`: gets the first element of the list. the numbering starts from 0 because of computer's base 2 counting standards.

In [ ]:
matching = (
    taxon_ranked_only.join(
        other=pl.LazyFrame(bos_cleaned),
        on=["genericName", "specificEpithet", "infraspecificEpithet"],
        how="right",
    )
    .rename({"taxonID": "matched_taxonID"})
    .select(
        "speciesId",
        "matched_taxonID",
        "acceptedNameUsageID",
        "taxonName",
        "domain",
        "kingdom",
        "phylum",
        "class",
        "subclass",
        "superorder",
        "order",
        "sub-order",
        "infraorder",
        "section",
        "subsection",
        "superfamily",
        "family",
        "subfamily",
        "tribe",
        "genus",
        "subgenus",
        "genericName",
        "specificEpithet",
        "infraspecificEpithet",
    )
    .with_columns(
        pl.when(pl.col("infraspecificEpithet") == "")
        .then(pl.lit(None))
        .otherwise("infraspecificEpithet")
        .alias("infraspecificEpithet")
    )
)
# matching and contetnious split
# What makes a data point contentious is where it has duplicate speciesId.
matching = matching.filter(~pl.col("speciesId").is_duplicated()).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(None))
    .cast(pl.Int64)
)
contentious = matching.filter(
    (pl.col("speciesId").is_duplicated())
).with_columns(
    acceptedNameUsageID=pl.col("acceptedNameUsageID")
    .fill_null(pl.lit(None))
    .cast(pl.Int64)
)

unique_contentious = contentious.filter(
    (pl.col("acceptedNameUsageID").is_null())
    & (
        pl.col("matched_taxonID").is_in(
            pl.col("acceptedNameUsageID").implode()
        )
    )
)

unique_contentious_speciesId = (
    unique_contentious.select("speciesId").collect().to_series().implode()
)  # Just the speciesIds
contentious = contentious.filter(
    ~pl.col("speciesId").is_in(unique_contentious_speciesId)
)  # Removing unique conentious

matching = pl.concat(
    [matching, unique_contentious],
)

matching = matching.with_columns(
    genus=pl.when(
        (pl.col("genus").is_null()) & (pl.col("specificEpithet").is_not_null())
    )
    .then(pl.col("taxonName").str.split(" ").list[0])
    .otherwise("genus")
)

no_match = (
    matching.filter(pl.col("matched_taxonID").is_null())
    .with_columns(
        taxonRank=pl.lit("BOSuncornirmedSpecies"),
        taxonomicStatus=pl.lit("BOSunformired"),
        parentNameUsageID=pl.lit(None),
    )
    .fill_null("")
    .collect()
)

# First Matching

`sort()`: sort by a specific column(s) in ascending order

In [ ]:
repeated_accepted_taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(pl.col("taxonomicStatus") == pl.lit("accepted"))
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .filter(~pl.col("canonicalName").is_null())
    .filter(pl.col("canonicalName") != "")
    .sort("canonicalName")
    .filter(pl.col("canonicalName").is_duplicated())
)

In [ ]:
priority_columns = [
    "infraspecificEpithet",
    "specificEpithet",
    "genus",
    "family",
    "order",
    "class",
    "phylum",
    "kingdom",
]

`vstack()`: Stack the new dataframe vertically.
`group_by()`: group the data by some column(s) and then do an operation such as `mean()`, `len()`, `agg()` and etc.
`agg()`: a function to work on the groups.

This line below is used to join the strings in the group using the delimiter ", "
`.agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))`

`split()`: used to split a string type column into a list of strings.

In [ ]:
_schema = {
    "feature_that_is_equal_to_canonicalName": pl.String,
    "matches": pl.String,
}
_r = (repeated_accepted_taxons).collect()
RAT_interim = pl.DataFrame(schema=_schema)
for _col in priority_columns:
    _canonical_names = (
        _r.filter(pl.col("canonicalName") == pl.col(_col))
        .select("canonicalName")
        .unique()
        .to_series()
        .to_list()
    )
    if len(_canonical_names) != 0:
        _row = pl.DataFrame(
            data={
                "feature_that_is_equal_to_canonicalName": _col,
                "matches": _canonical_names,
            },
            schema=_schema,
        )
        RAT_interim = RAT_interim.vstack(_row)
RAT_feats = (
    RAT_interim.group_by("matches")
    .agg(pl.col("feature_that_is_equal_to_canonicalName").str.join(", "))
    .with_columns(
        pl.col("feature_that_is_equal_to_canonicalName").str.split(", ")
    )
    .sort("feature_that_is_equal_to_canonicalName")
)

`copy()`: deep copy, you may search it up to read more about it.
`reverse()`: reverse a list in place
`index()`: The index() method in Python is used to find the position of the first occurrence of a specified value in a list or string.

`bool()`: cast a value into boolean
`None`: Null value in python
`append()`: add new element to the back of a list
`len()`: get length of string,list,etc
`item()`: only in polars Series where there is one value, this will get that value out.

This code returns a boolean `pl.Series`:
```
  (
                        _no_match_subset_to_update[_col1].item()
                        == taxon_data_to_select_from[
                            int(not bool(selected_row_int)), :
                        ]
                    )
                    ```

# I stopped the documentation here as I was given the instruction that these instrucitons are no longer needed.

In [ ]:
updated_to_matching = []
still_no_match = []
_collected_repeated_taxons = repeated_accepted_taxons.collect().fill_null("")
_reversed_priority_columns = priority_columns.copy()
_reversed_priority_columns.reverse()
for _col in _reversed_priority_columns:
    for _match in RAT_feats["matches"]:
        # skipping those that are not in RAT_feats
        if _match not in no_match[_col].to_list():
            continue

        taxon_data_to_select_from = _collected_repeated_taxons.filter(
            pl.col("canonicalName") == _match
        ).select(["taxonID"] + _reversed_priority_columns)

        # Select from the two rows what taxon data to select.
        selected_row_int = -1
        for i, _tuple in enumerate(
            taxon_data_to_select_from.select(
                ["taxonID"] + priority_columns
            ).iter_rows()
        ):
            col_index = priority_columns.index(_col)
            _x = _tuple[col_index]
            if not bool(_x):
                selected_row_int = i  # Since there are usually only two rows from taxon_data_to_select_from, the results is either 0 or 1
                break
        assert selected_row_int != -1

        # has_predicament1 is when kingdom to genus all has values and it is kind of leveled between the two rows of the taxon data.
        has_predicament1 = False
        _l = taxon_data_to_select_from["family"].to_list()
        if _col == "genus" and _l[0] is not None and _l[1] is not None:
            has_predicament1 = True

        chosen_taxonId = taxon_data_to_select_from[selected_row_int, 0]
        other_taxonId = taxon_data_to_select_from[
            int(
                not bool(selected_row_int)
            ),  # converts the int value into boolean than reverse the boolean before converting back to int.
            0,
        ]

        # Getting the _no_match_subset_to_update section
        _no_match_subset_to_update = no_match.filter(
            pl.col(_col) == _match,
        )
        _x = _reversed_priority_columns[:-3]
        feature_to_find_nulls = (
            _x[_reversed_priority_columns.index(_col) + 1]
            if len(_x) > _reversed_priority_columns.index(_col) + 1
            else None
        )  # just the next feature to find null
        if feature_to_find_nulls is not None:
            _no_match_subset_to_update = _no_match_subset_to_update.filter(
                pl.col(feature_to_find_nulls) == ""
            )
            if _no_match_subset_to_update.is_empty():
                still_no_match_subset = no_match.filter(
                    pl.col(_col) == _match,
                ).with_columns(
                    current_feature=pl.lit(_col), current_name=pl.lit(_match)
                )
                still_no_match.append(still_no_match_subset)

                continue

        # Settling predicament one
        # this predicament thing should only happen when the subset is of only 1 row.
        if has_predicament1 and _no_match_subset_to_update.shape[0] == 1:
            match = True
            for _col1 in ["class", "order", "family"]:
                match *= (
                    _no_match_subset_to_update[_col1].item()
                    == taxon_data_to_select_from[selected_row_int, :][
                        _col1
                    ].item()
                )
            if not match:
                match = True
                for _col1 in ["class", "order", "family"]:
                    match *= (
                        _no_match_subset_to_update[_col1].item()
                        == taxon_data_to_select_from[
                            int(not bool(selected_row_int)), :
                        ][_col1].item()
                    )  # searching in the three columns of interests for matches
                assert (
                    match
                )  # Double check that there is no unmatch.  # noqa: E712
                temp = chosen_taxonId
                chosen_taxonId = other_taxonId
                other_taxonId = temp

        # turning these rows to matching by filling parentNameUsageID
        _no_match_subset_to_update = _no_match_subset_to_update.with_columns(
            parentNameUsageID=pl.when(
                (pl.col("infraspecificEpithet").is_null())
                & (pl.col("specificEpithet").is_not_null())
                & (pl.col("genus").is_not_null())
            )
            .then(pl.lit(other_taxonId))
            .otherwise(pl.lit(chosen_taxonId))
        )

        updated_to_matching.append(_no_match_subset_to_update)

# turn them into dataframes and writing to csv files
updated_to_matching = pl.concat(
    updated_to_matching, rechunk=True, parallel=True
)
updated_to_matching.write_csv("first_matches_set_from_wrangling.csv")

still_no_match = pl.concat(still_no_match, rechunk=True, parallel=True)
still_no_match = still_no_match.join(
    updated_to_matching.select("speciesId"), on="speciesId", how="anti"
)

# Second Matching Wrangle

In [ ]:
taxons = (
    pl.scan_csv("gbif/Taxon.tsv", separator="\t", quote_char=None, cache=True)
    .filter(
        pl.col("taxonomicStatus") == pl.lit("accepted"),
        pl.col("taxonRank") != "unranked",
    )
    .filter(pl.col("kingdom").is_in(["Animalia", "Plantae"]))
    .select(["taxonID"] + priority_columns)
)


def join_and_parentId_insertion(
    match_on: pl.DataFrame, _taxons_subset: pl.DataFrame, on: list
) -> pl.DataFrame:
    return (
        match_on.join(_taxons_subset, on=on)
        .with_columns(
            parentNameUsageID=pl.when(pl.col("taxonID").is_not_null())
            .then("taxonID")
            .otherwise("parentNameUsageID")
        )
        .drop("taxonID")
    )

## Non nulls stopped at class

In [ ]:
_filter_class = (
    pl.col("class") != "",
    pl.col("order") == "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
    pl.col("current_feature") == "phylum",
)
match_on_class = still_no_match.with_columns(
    name_to_match=pl.when(_filter_class).then("class").otherwise(None)
).filter(_filter_class)

_x_class = (
    still_no_match.filter(_filter_class)
    .select("class")
    .group_by("class")
    .len()
)

_taxons_subset_class = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_null(),
        pl.col("genus").is_null(),
        pl.col("class").is_in(_x_class["class"].unique().to_list()),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        # pl.col("phylum").is_in(_x_class["phylum"].unique().to_list()),
    )
    .select("taxonID", "class")
    .collect()
)
match_on_class = join_and_parentId_insertion(
    match_on_class, _taxons_subset_class, ["class"]
)

## non nulls stopping at order

In [ ]:
_filter_order = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") == "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
    pl.col("current_feature") == "phylum",
)
match_on_order = still_no_match.with_columns(
    name_to_match=pl.when(_filter_order).then("order").otherwise(None)
).filter(_filter_order)
_x_order = (
    still_no_match.filter(_filter_order)
    .select("order")
    .group_by("order")
    .len()
)

_taxons_subset_order = (
    taxons.filter(
        pl.col("family").is_null(),
        pl.col("order").is_in(_x_order["order"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
    )
    .select("taxonID", "order")
    .collect()
)
match_on_order = join_and_parentId_insertion(
    match_on_order, _taxons_subset_order, ["order"]
)

## non nulls stopping at family

In [ ]:
_filter_family = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") == "",
    pl.col("specificEpithet") == "",
    pl.col("infraspecificEpithet") == "",
)
match_on_family = still_no_match.with_columns(
    name_to_match=pl.when(_filter_family).then("family").otherwise(None)
).filter(_filter_family)
_x_family = (
    still_no_match.filter(_filter_family)
    .select("family", "order")
    .group_by("family", "order")
    .len()
)

_taxons_subset_family = (
    taxons.filter(
        pl.col("family").is_in(_x_family["family"].unique().to_list()),
        pl.col("genus").is_null(),
        pl.col("specificEpithet").is_null(),
        pl.col("infraspecificEpithet").is_null(),
        pl.col("order").is_in(_x_family["order"].unique().to_list()),
    )
    .select("taxonID", "family")
    .collect()
)
match_on_family = join_and_parentId_insertion(
    match_on_family, _taxons_subset_family, ["family"]
)

## non nulls stopping at specificEpithet

In [ ]:
_filter_specificEpithet = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("specificEpithet") != "",
    pl.col("infraspecificEpithet") == "",
)

to_be_match_on_specificEpithet = still_no_match.filter(_filter_specificEpithet)

_x_specificEpithet = (
    still_no_match.filter(_filter_specificEpithet)
    .select("specificEpithet", "genus")
    .group_by("specificEpithet", "genus")
    .len()
)

_taxons_subset_specificEpithet = (
    taxons.filter(
        pl.col("specificEpithet").is_in(
            _x_specificEpithet["specificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(_x_specificEpithet["genus"].unique().to_list()),
    )
    .select("taxonID", "specificEpithet", "genus")
    .collect()
)
match_on_specificEpithet = join_and_parentId_insertion(
    to_be_match_on_specificEpithet,
    _taxons_subset_specificEpithet,
    ["specificEpithet", "genus"],
)

## non nulls stopping at infraspecificEpithet

In [ ]:
_filter_infraspecificEpithet = (
    pl.col("class") != "",
    pl.col("order") != "",
    pl.col("family") != "",
    pl.col("genus") != "",
    pl.col("infraspecificEpithet") != "",
)  # different filter because some don't have specificEpithet either, if I am not wrong
to_be_match_on_infraspecificEpithet = still_no_match.filter(
    _filter_infraspecificEpithet
)

_x_infraspecificEpithet = (
    still_no_match.filter(_filter_infraspecificEpithet)
    .select("infraspecificEpithet", "genus")
    .group_by("infraspecificEpithet", "genus")
    .len()
)

_taxons_subset_infraspecificEpithet = (
    taxons.filter(
        pl.col("infraspecificEpithet").is_in(
            _x_infraspecificEpithet["infraspecificEpithet"].unique().to_list()
        ),
        pl.col("genus").is_in(
            _x_infraspecificEpithet["genus"].unique().to_list()
        ),
    )
    .select("taxonID", "infraspecificEpithet", "genus")
    .collect()
)

match_on_infraspecificEpithet = join_and_parentId_insertion(
    to_be_match_on_infraspecificEpithet,
    _taxons_subset_infraspecificEpithet,
    ["infraspecificEpithet", "genus"],
)

## Combining all these second matching wrangling stage dataframes of the ranks

In [ ]:
_to_drop = ["current_feature", "current_name", "name_to_match"]
_to_drop2 = ["current_feature", "current_name"]
_new_match = pl.concat(
    [
        match_on_class.drop(_to_drop),
        match_on_order.drop(_to_drop),
        match_on_family.drop(_to_drop),
        match_on_specificEpithet.drop(_to_drop2),
        match_on_infraspecificEpithet.drop(_to_drop2),
    ]
)
# print("second wrangle new matches", _new_match.shape)
_new_match.write_csv("second_matches_set_from_wrangling.csv")
_new_match = pl.concat(
    [
        _new_match,
        updated_to_matching.with_columns(
            pl.col("parentNameUsageID").cast(pl.Int64)
        ),
    ]
)
_new_match.write_csv("updated_to_matching.csv")
_no_match = no_match.join(
    _new_match.select("speciesId"), on="speciesId", how="anti"
)
_no_match.write_csv("no_match.csv")